In [1]:
import sys
sys.path.append('/home/SharedData/anaconda2/external/fastai/')

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
!pip install bcolz

In [4]:
from fastai.imports import *

In [5]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [6]:
from nn_model import MLCNN

In [9]:
from cfg import process_config
config = process_config('config_mlcnn.json')

In [22]:
# Data loading
ptb_tdir = Path(config.data_dir)
ptb_data_tdir = ptb_tdir / 'data'
ptb_tdir_str = str(ptb_tdir / 'data') + '/'
patient_list_file = str(config.patient_file)
control_list_file = str(config.control_file)
positive_list_file = str(config.positive_file)

with open(patient_list_file, 'r') as f:
    patient_list = f.read().splitlines()
with open(control_list_file, 'r') as f:
    control_list = f.read().splitlines()
with open(positive_list_file, 'r') as f:
    positive_list = f.read().splitlines()

In [12]:
patient_list_pd = pd.DataFrame(patient_list)
control_list_pd = pd.DataFrame(control_list)
positive_list_pd = pd.DataFrame(positive_list)

In [16]:
import wfdb

In [23]:
!ls {ptb_data_tdir}

patient001  patient050	patient099  patient151	patient201  patient250
patient002  patient051	patient100  patient152	patient202  patient251
patient003  patient052	patient101  patient153	patient203  patient252
patient004  patient053	patient102  patient154	patient204  patient253
patient005  patient054	patient103  patient155	patient205  patient254
patient006  patient055	patient104  patient156	patient206  patient255
patient007  patient056	patient105  patient157	patient207  patient256
patient008  patient057	patient106  patient158	patient208  patient257
patient009  patient058	patient107  patient159	patient209  patient258
patient010  patient059	patient108  patient160	patient210  patient259
patient011  patient060	patient109  patient162	patient211  patient260
patient012  patient061	patient110  patient163	patient212  patient261
patient013  patient062	patient111  patient164	patient213  patient262
patient014  patient063	patient112  patient165	patient214  patient263
patient015  patient064	patient113 

In [34]:
def one_to_one_map(inp):
    label_dict = json.load('label_dict.json')
    return label_dict[inp]

In [35]:
p1 = patient_list[0]
sig, fields = wfdb.rdsamp(ptb_tdir_str + p1)
fields['comments'][4]


'Reason for admission: Myocardial infarction'

In [17]:
some_dict = {}
for p in patient_list:
    _, fields = wfdb.rdsamp(ptb_tdir_str + p)

patient001/s0010_re
patient001/s0014lre
patient001/s0016lre
patient002/s0015lre
patient003/s0017lre
patient004/s0020are
patient004/s0020bre
patient005/s0021are
patient005/s0021bre
patient005/s0025lre
patient005/s0031lre
patient005/s0101lre
patient006/s0022lre
patient006/s0027lre
patient006/s0064lre
patient007/s0026lre
patient007/s0029lre
patient007/s0038lre
patient007/s0078lre
patient008/s0028lre
patient008/s0037lre
patient008/s0068lre
patient009/s0035_re
patient010/s0036lre
patient010/s0042lre
patient010/s0061lre
patient011/s0039lre
patient011/s0044lre
patient011/s0049lre
patient011/s0067lre
patient012/s0043lre
patient012/s0050lre
patient013/s0045lre
patient013/s0051lre
patient013/s0072lre
patient014/s0046lre
patient014/s0056lre
patient014/s0071lre
patient015/s0047lre
patient015/s0057lre
patient015/s0152lre
patient016/s0052lre
patient016/s0060lre
patient016/s0076lre
patient017/s0053lre
patient017/s0055lre
patient017/s0063lre
patient017/s0075lre
patient018/s0054lre
patient018/s0059lre


In [13]:
patient_list_pd

,0
0,patient001/s0010_re
1,patient001/s0014lre
2,patient001/s0016lre
3,patient002/s0015lre
4,patient003/s0017lre
5,patient004/s0020are
6,patient004/s0020bre
7,patient005/s0021are
8,patient005/s0021bre
9,patient005/s0025lre
